In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Logit PAG

GenerationMixin.generate 이 좀 어질어질하네.

되도록 건드리지 않는 것을 추구해보자

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# define hook
from pag.utils import get_module_by_name

def perturb_attn_map(module, input, kwargs, output, inv_temp:float = 1.0, guidance_scale:float = 7.5):
    layer_past = kwargs['layer_past']
    use_cache = kwargs['use_cache']
    head_mask = kwargs['head_mask']
    attention_mask = kwargs['attention_mask']
    output_attentions = kwargs['output_attentions']

    hidden_states = input[0]
    query, key, value = module.c_attn(hidden_states).split(module.split_size, dim=2)

    query = module._split_heads(query, module.num_heads, module.head_dim)
    key = module._split_heads(key, module.num_heads, module.head_dim)
    value = module._split_heads(value, module.num_heads, module.head_dim)

    if layer_past is not None:
        past_key, past_value = layer_past
        key = torch.cat((past_key, key), dim=-2)
        value = torch.cat((past_value, value), dim=-2)

    if use_cache is True:
        present = (key, value)
    else:
        present = None

    # original attn 
    if module.reorder_and_upcast_attn:
        attn_output_o, attn_weights_o = module._upcast_and_reordered_attn(query, key, value, attention_mask, head_mask)
    else:
        attn_output_o, attn_weights_o = module._attn(query, key, value, attention_mask, head_mask)

    attn_output_o = module._merge_heads(attn_output_o, module.num_heads, module.head_dim)
    attn_output_o = module.c_proj(attn_output_o)
    attn_output_o = module.resid_dropout(attn_output_o)
    
    # perturbed attn
    if module.reorder_and_upcast_attn:
        attn_output_p, attn_weights_p = module._upcast_and_reordered_attn(inv_temp * query, key, value, attention_mask, head_mask)
    else:
        attn_output_p, attn_weights_p = module._attn(inv_temp * query, key, value, attention_mask, head_mask)

    attn_output_p = module._merge_heads(attn_output_p, module.num_heads, module.head_dim)
    attn_output_p = module.c_proj(attn_output_p)
    attn_output_p = module.resid_dropout(attn_output_p)

    # guidance
    attn_output = attn_output_o + guidance_scale * (attn_output_o - attn_output_p)
    
    outputs = (attn_output, present)
    if output_attentions:
        outputs += (attn_weights_o,)

    return outputs  # a, present, (attentions)

### Eval

In [5]:
from datasets import load_dataset
ds = load_dataset('hails/mmlu_no_train', 'formal_logic')

/home/nsml/.local/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating test split: 126 examples [00:00, 770.29 examples/s]
Generating validation split: 14 examples [00:00, 2296.27 examples/s]
Generating dev split: 5 examples [00:00, 36.29 examples/s]


In [15]:
import numpy as np
from functools import partial

hook_layer_list = [
    # 'transformer.h.0.attn', 
    # 'transformer.h.1.attn', 
    'transformer.h.2.attn', 
    'transformer.h.3.attn', 
    'transformer.h.4.attn', 
    'transformer.h.5.attn', 
    # 'transformer.h.6.attn', 
    # 'transformer.h.7.attn', 
    # 'transformer.h.8.attn', 
    # 'transformer.h.9.attn', 
    # 'transformer.h.10.attn', 
    # 'transformer.h.11.attn', 
]

prompt = ds['test'][1]['question']

for guidance_scale in np.linspace(0, 1, 4):
    for inv_temp in [1.0, 1e-1, 1e-2]:
        # hook (make attn map identity)
        handles = []
        for layer_name in hook_layer_list:
            module = get_module_by_name(model, layer_name)
            perturb_attn_map_fn = partial(perturb_attn_map, inv_temp=inv_temp, guidance_scale=guidance_scale)
            handle = module.register_forward_hook(perturb_attn_map_fn, with_kwargs=True)
            handles.append(handle)

        tokenizer.pad_token_id = tokenizer.eos_token_id
        inputs = tokenizer([prompt], return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=64, return_dict_in_generate=True, output_scores=True)

        # generated result
        print()
        print(f"### Generated text: guidance_scale {guidance_scale}-inv_temp_{inv_temp}###")
        print(f'{tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)}')

        for handle in handles:
            handle.remove()
            
        if guidance_scale == 0.0:
            break

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text: guidance_scale 0.0-inv_temp_1.0###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text: guidance_scale 0.3333333333333333-inv_temp_1.0###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when


...


### Generated text: guidance_scale 0.6666666666666666-inv_temp_0.01###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The first step is to define the predicate logic.

The predicate logic is a function that takes a predicate and returns a list of values.

The predicate logic is a function that takes a predicate and returns a list of values. The predicate logic is a function that takes a predicate and returns a list


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text: guidance_scale 1.0-inv_temp_1.0###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when translating a predicate into a predicate logic.

The following is a list of the most common mistakes that people make when


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text: guidance_scale 1.0-inv_temp_0.1###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The first thing to do is to make sure that the first argument is a predicate.

defmodule foo.foo do |x| x = x.x | x.x = x.x defmodule bar.bar do |x| x.x = x.x defmodule foo.bar

### Generated text: guidance_scale 1.0-inv_temp_0.01###
Select the best translation into predicate logic. David teaches Chris. (c: Chris; d: David; Txy: x teaches y)

The first thing to do is to make sure that the first argument is a predicate.

defmodule foo.foo do |x| x.x = x.x defmodule bar.bar do |x| x.x = x.x defmodule foo.bar do |x| x.


In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(["Today is"], return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=15, return_dict_in_generate=True, output_scores=True)

print()
print(f'{tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Today is the day when we can all be proud of our country and our values.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Today is the day when we can all be proud of our country and our values.


In [25]:
kwargs = {'layer_past': None, 'attention_mask': torch.tensor([[[[-0., -0.]]]]), 'head_mask': None, 'use_cache': True, 'output_attentions': False}

for k, v in kwargs.items():
    locals()[k] = v

print(layer_past)

None


In [15]:
>>> tensor1 = torch.randn(10, 16, 3, 4)
>>> tensor2 = torch.randn(10, 16, 4, 5)
>>> torch.matmul(tensor1, tensor2).size()

torch.Size([10, 16, 3, 5])

In [48]:
# import matplotlib.pyplot as plt
# plt.plot(guidance_traj[4].flatten())

In [ ]:

# Attention map 수정을 위한 모델 복사
model_modified = AutoModelForCausalLM.from_pretrained(model_name)

# Attention map을 수정할 layer 번호 설정
modify_layers = [2, 4]

# Attention map을 identity matrix로 설정하는 함수
def modify_attention(module, input, output):
    # module이 MultiheadAttention인 경우에만 수정
    if isinstance(module, torch.nn.MultiheadAttention):
        # layer 번호 가져오기
        layer_num = int(module.layer_num)
        # 수정할 layer인 경우
        if layer_num in modify_layers:
            attention_probs = output[1]
            batch_size, num_heads, seq_length, _ = attention_probs.shape
            print(f"Layer {layer_num} - Attention shape: {attention_probs.shape}")
            # Attention shape 확인
            assert attention_probs.shape[-1] == seq_length, f"Attention shape mismatch: {attention_probs.shape}"
            # Attention map을 identity matrix로 설정
            attention_probs.data = torch.eye(seq_length).expand(batch_size, num_heads, -1, -1)
    return output

# Hook 등록 함수
def register_hooks(model):
    # 모든 module에 대해 iterate
    for name, module in model.named_modules():
        # module이 MultiheadAttention인 경우 hook 등록
        if isinstance(module, torch.nn.MultiheadAttention):
            layer_num = name.split(".")[2]
            module.layer_num = layer_num
            module.register_forward_hook(modify_attention)

# 수정된 모델에 hook 등록
register_hooks(model_modified)

In [7]:
# 텍스트 생성 함수
def generate_text(input_text, guidance_scale):
    # 입력 텍스트 인코딩
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)
    
    # Attention map 수정 후 logit_p 계산
    with torch.no_grad():
        outputs = model_modified(input_ids, attention_mask=attention_mask)
        logit_p = outputs.logits
        
    # 원래 attention으로 logit_o 계산
    with torch.no_grad():
        outputs = model_original(input_ids, attention_mask=attention_mask)
        logit_o = outputs.logits
        
    # logit 계산
    logit = logit_o + guidance_scale * (logit_o - logit_p)
    print(f"logit shape: {logit.shape}")
    print(f"logit: {logit}")
    
    # 텍스트 생성
    output = model_original.generate(
        input_ids,
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=attention_mask,
        logits_processor=[
            lambda input_ids, scores: scores + guidance_scale * (scores - logit[:, -1, :])
        ],
        output_scores=True,
        return_dict_in_generate=True,
        top_p=0.9
    )
    
    print(f"Generated text: {tokenizer.decode(output.sequences[0], skip_special_tokens=True)}")
    print(f"Processed logits: {output.scores[0].shape}")
    print(f"Processed logits: {output.scores[0]}")
    
    return tokenizer.decode(output.sequences[0], skip_special_tokens=True)

In [8]:
# 텍스트 생성 및 출력
input_text = "Hello, how are you? I'm doing well."
guidance_scale = 6.0
generated_text = generate_text(input_text, guidance_scale)
print("Generated Text:")
print(generated_text)

logit shape: torch.Size([1, 11, 50257])
logit: tensor([[[ -35.2362,  -35.3266,  -38.9754,  ...,  -44.4645,  -43.9975,
           -36.4580],
         [-112.6171, -114.5832, -116.5724,  ..., -119.0128, -118.8059,
          -111.6917],
         [-116.7137, -117.5931, -123.1624,  ..., -125.6588, -125.2527,
          -119.3150],
         ...,
         [-107.5247, -109.3616, -113.5464,  ..., -115.4737, -118.1396,
          -112.0031],
         [ -82.1815,  -84.9542,  -93.1059,  ...,  -98.1118,  -98.1967,
           -88.8256],
         [-147.8117, -146.8510, -149.7308,  ..., -160.3841, -160.5273,
          -143.6174]]])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated text: Hello, how are you? I'm doing well. I intend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend
Processed logits: torch.Size([1, 50257])
Processed logits: tensor([[-147.8117, -146.8510, -149.7308,  ..., -160.3841, -160.5273,
         -143.6174]])
Generated Text:
Hello, how are you? I'm doing well. I intend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend tremend


In [9]:
# Perplexity 계산 함수
def calculate_perplexity(model_original, model_modified, tokenizer, text, guidance_scale):
    encoded_text = tokenizer.encode(text, return_tensors='pt')
    attention_mask = torch.ones_like(encoded_text)
    
    with torch.no_grad():
        outputs_original = model_original(encoded_text, attention_mask=attention_mask, labels=encoded_text)
        loss_original = outputs_original.loss
        
        outputs_modified = model_modified(encoded_text, attention_mask=attention_mask)
        logits_modified = outputs_modified.logits
        
        logits = outputs_original.logits + guidance_scale * (outputs_original.logits - logits_modified)
        loss_modified = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), encoded_text.view(-1))
        
        perplexity = torch.exp((loss_original + loss_modified) / 2)
    
    return perplexity.item()

In [10]:
# Perplexity 계산 및 출력
perplexity = calculate_perplexity(model_original, model_modified, tokenizer, generated_text, guidance_scale)
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 535.28
